In [ ]:
from collections import defaultdict
import json
from itertools import chain
import os
import logging
import sys
from pathlib import Path
import re
from typing import Dict, List, NamedTuple, Tuple

import django
from django.db.models import Model, QuerySet
from django.db.models.signals import post_save
from django.db import transaction
import pandas as pd

In [ ]:
from pathlib import Path
from typing import Dict
from typing import List, NamedTuple

import pandas as pd
from openpyxl import load_workbook

In [ ]:
ROOT = Path('~/dev/temp/hawc-pfas/project').expanduser()
sys.path.append(ROOT)
os.chdir(ROOT)

os.environ["DJANGO_SETTINGS_MODULE"] = "hawc.settings.local"
django.setup()
logger = logging.getLogger(__name__)

In [ ]:
from summary.models import DataPivotQuery, Visual
from study.models import Study

In [ ]:
assessment_id = 100000026
dp_mapping = {
    el['slug']: el['id'] for el in
    DataPivotQuery.objects.filter(assessment_id=assessment_id).values('id', 'slug')
}

In [ ]:
class ImportData(NamedTuple):
    chem: str
    studies: List[str]
    visuals: List[str]
    data_pivots: List[str]

In [ ]:
def get_import_data(fn: str, ws: str, chem_rng: str, studies_rng: List[str], viz_rng: List[str]) -> Dict:
    wb = load_workbook(filename=fn, read_only=True)
    ws = wb[ws]
    
    chem = ws[chem_rng].value
    
    studies = []
    for study in studies_rng:
        studies.extend([cell[0].value for cell in ws[study]])        
    
    regex = re.compile(r'/study/(\d+)/')
    studies = list(set([int(regex.findall(url)[0]) for url in studies]))
    
    studies_found = list(
        Study.objects.filter(assessment_id=assessment_id, id__in=studies).values_list('id', flat=True)
    )
    if len(studies_found) != len(studies):
        for study in (set(studies) - set(studies_found)):
            print(f'Missing study: https://hawcprd.epa.gov/study/{study}/')                   
    studies = sorted(studies_found)
    
    visuals = []
    for visual in viz_rng:
        visuals.extend([cell[0].value for cell in ws[visual]])    
                
    viz = [el for el in visuals if '/visual/' in el]
    regex = re.compile(r'/summary/visual/(\d+)/')
    viz = list(set([int(regex.findall(url)[0]) for url in viz]))
        
    viz_found = list(
        Visual.objects.filter(assessment_id=assessment_id, id__in=viz).values_list('id', flat=True)
    )
    if len(viz_found) != len(viz):
        for v in (set(viz) - set(viz_found)):
            print(f'Missing visual: https://hawcprd.epa.gov/summary/visual/{v}/')                   
    viz = sorted(viz_found)
    
    
    dp = [el for el in visuals if '/data-pivot/' in el]
    regex = re.compile(rf'/summary/data-pivot/assessment/{assessment_id}/(.+)/')
    slugs = [regex.findall(url)[0] for url in dp]
    dp_ids = [dp_mapping.get(slug, -1) for slug in slugs]
    for id_, slug in zip(dp_ids, slugs):
        if id_ == -1:
            print(f'Missing data pivot: https://hawcprd.epa.gov/summary/data-pivot/assessment/{assessment_id}/{slug}/')                       
            
    dp_ids = sorted(list(set([dp_id for dp_id in dp_ids if dp_id>0])))
    
    
    wb.close()
    return ImportData(chem=chem, studies=studies, visuals=viz, data_pivots=dp_ids)

In [ ]:
pfhxa = get_import_data(
    Path('~/Desktop/tofile/pfas-migrations/PFHxA HAWC migration MA 191209 v2.xlsx').expanduser(), 
    "Sheet1", 
    "E2", 
    studies_rng=["A2:A34"], 
    viz_rng=["C2:C34"]
)
pfhxa

In [ ]:
pfna = get_import_data(
    Path('~/Desktop/tofile/pfas-migrations/PFNA MIGRATION TEMPLATE FOR ANDY SHAPIRO_MT-ER.xlsx').expanduser(), 
    "Sheet1", 
    "E2", 
    studies_rng=["A3:A33", "A36:A38", "A41:A244"], 
    viz_rng=["C3:C14", "C17:C24", "C27:C35", "C38:C38"]
)
pfna

In [ ]:
pfda = get_import_data(
    Path('~/Desktop/tofile/pfas-migrations/PFDA HAWC data migration_v2.xlsx').expanduser(), 
    "Sheet1", 
    "E2", 
    studies_rng=["A3:A23", "A27:A145"], 
    viz_rng=["C2:C68"]
)
pfda

In [ ]:
pfba = get_import_data(
    Path('~/Desktop/tofile/pfas-migrations/PFBA MIGRATION TEMPLATE FOR ANDY SHAPIRO_MT draft.xlsx').expanduser(), 
    "Sheet1", 
    "E2", 
    studies_rng=["A3:A10","A17:A25"], 
    viz_rng=["C2:C36", "C41:C42", "C47:C47"]
)
pfba

In [ ]:
pfhxs = get_import_data(
    Path('~/Desktop/tofile/pfas-migrations/HAWC Migration.xlsx').expanduser(), 
    "Sheet1", 
    "E2", 
    studies_rng=["A2:A204"], 
    viz_rng=["C2:C25", "C28:C31", "C33:C40"]
)
pfhxs